In [ ]:
This week will be officially the final week for NLP study.

1. Write a summary of each topic.
1) Normal mode for transformer
2) teacher forcing
3) Scheduled sampling
4) GAN. Specifically for its architecture from transformer, which plays a role of generator and discriminator in transformer for decoder only model?
(adapt layer part for fine tuning.)
5) Why do we choose decoder only model over encoder only model for text generation?


2. We will be training a scratch model built with pytorch and transformer decoder layers included with three methods; normal mode for next token prediction, teacher forcing, and scheduled sampling . Then, we will fine tune with reddit dataset between body(input) and comments(target). 
Lastly, We will fine-tune on phi-1 model by MS and compare the result.
Please pick a small dataset around 100-200 sentences.
You can use any tokenizer.

In [6]:
# %pip uninstall transformers
# %pip install git+https://github.com/zphang/transformers@llama_push
# %pip install accelerate
# %pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers
# %pip install git+https://github.com/huggingface/transformers
# %pip install -U jinja2
# %pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      /private/var/folders/td/1zp1k2xj2m5dwylk8w2lttg80000gn/T/pip-install-v8y1sqcy/flash-attn_eac960510baf480a873aebb4c7fd5b4f/setup.py:78: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/td/1zp1k2xj2m5dwylk8w2lttg80000gn/T/pip-install

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import transformers
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
torch.__version__

'2.2.1'

In [4]:
df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/tchun@carparts.com/CNN_Articels_clean_2__1_-1.csv").toPandas()

NameError: name 'spark' is not defined

In [7]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="microsoft/phi-2", torch_dtype=torch.bfloat16, device_map="auto",trust_remote_code=True)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [ ]:
MODEL_NAME = 'microsoft/phi-2'
config_m = transformers.AutoConfig.from_pretrained(MODEL_NAME)

In [ ]:
config_m.task_specific_params = {'text-generation':{'do_sample':True,'max_length':50}}

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_NAME)#,config=config_m)
model.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Out[7]: PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (1): PhiDecoderLayer(
        (self_attn)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME,config=config_m)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class MyDset(torch.utils.data.Dataset):
  """A custom dataset"""
  def __init__(self, data: list[str]):
    self.data = []
    for d in data:
      input_ids = torch.tensor(d, dtype=torch.int64)
      attention_mask = torch.ones(len(d), dtype=torch.int64)
      self.data.append({'input_ids': input_ids,
      'attention_mask': attention_mask, 'labels': input_ids})

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx: int):
    return self.data[idx]


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
class GEN_Dataset(torch.utils.data.Dataset):

  def __init__(self, txt_list, tokenizer, max_length=64):

    self.tokenizer = tokenizer
    self.txt_list = txt_list
    self.max_length = max_length
    # self.input_ids = 
    # self.attn_masks = []

    # for txt in txt_list:

    #   encodings_dict = tokenizer(txt.split(' '), truncation=True, max_length=max_length, padding="max_length")

    #   self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
    #   self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.txt_list)

  def __getitem__(self, idx):
    tokenized = tokenizer(self.txt_list[idx], truncation=True, max_length=self.max_length, padding="max_length",return_tensors='pt')
    tokenized['labels']= \
    torch.cat((
      tokenized['input_ids'][:,1:] , torch.tensor([[tokenizer.pad_token_id]])
      
    ),dim=1)
    return tokenized

In [ ]:
# torch.tensor.cat([tokenizer('hi',return_tensors='pt')['input_ids'],])

tokenizer.pad_token_id

Out[22]: 50295

In [ ]:
tokenizer('hi yes',return_tensors='pt')['input_ids'][:,1:]

Out[41]: tensor([[3763]])

In [ ]:
torch.cat(
  (tokenizer('hi yes',return_tensors='pt')['input_ids'][1:],torch.tensor([[tokenizer.pad_token_id]]))
                                                                                 ,dim=1)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
File <command-1754353213322319>:1
----> 1 torch.cat(
      2   (tokenizer('hi yes',return_tensors='pt')['input_ids'][1:],torch.tensor([[tokenizer.pad_token_id]]))
      3                                                                                  ,dim=1)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 0 but got size 1 for tensor number 1 in the list.

In [ ]:
from itertools import chain
list_train_text = df.dropna()['Article'][:100].to_list()
list_train_text = list(chain.from_iterable([i.split('.') for i in list_train_text]))
list_train_text = [' '.join(i.split(' ')[:7]) for i in list_train_text]
data_set = GEN_Dataset(txt_list=list_train_text,tokenizer=tokenizer)
data_set
train_size = int(0.9 * data_set.__len__())
val_size = data_set.__len__() - train_size
train_dataset, val_dataset = torch.utils.data.random_split(data_set, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  592 training samples
   66 validation samples


In [ ]:

train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            sampler = torch.utils.data.RandomSampler(train_dataset), 
            batch_size = 5
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = torch.utils.data.DataLoader(
            val_dataset, # The validation samples.
            sampler = torch.utils.data.SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = 1 # Evaluate with this batch size.
        )

In [ ]:
# some parameters I cooked up that work reasonably well

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [ ]:
model.to('cpu')

Out[46]: PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (1): PhiDecoderLayer(
        (self_attn

In [ ]:
optimizer = transformers.AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-6242db26-4d77-48de-92e8-6c9c27ea84f6/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import datetime
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

Out[18]: PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (1): PhiDecoderLayer(
        (self_attn

In [ ]:
import time,random
device= 'cpu'
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch['input_ids'].to(device).squeeze()
        b_labels = batch['labels'].to(device).squeeze()
        b_masks = batch['attention_mask'].to(device).squeeze()

        model.zero_grad()        
        
        outputs = model(  input_ids=b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        del b_input_ids
        del b_labels
        del b_masks
        torch.cuda.empty_cache()
        loss.backward()

        optimizer.step()

        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


======== Epoch 1 / 5 ========
Training...

  Average training loss: 16.04
  Training epoch took: 0:30:50

======== Epoch 2 / 5 ========
Training...

  Average training loss: 16.04
  Training epoch took: 0:29:19

======== Epoch 3 / 5 ========
Training...

  Average training loss: 16.04
  Training epoch took: 0:28:28

======== Epoch 4 / 5 ========
Training...

  Average training loss: 16.04
  Training epoch took: 0:28:07

======== Epoch 5 / 5 ========
Training...

  Average training loss: 16.04
  Training epoch took: 0:28:03


In [ ]:
# del b_input_ids
# del b_labels
# del b_masks
torch.cuda.empty_cache()

In [ ]:
training_args = transformers.TrainingArguments(
output_dir="idiot_save/",
learning_rate=1e-3,
per_device_train_batch_size=1,
per_device_eval_batch_size=1,
num_train_epochs=5,
evaluation_strategy='epoch',
save_strategy='no',
)
trainer = transformers.Trainer(
 model=model,
 args=training_args,
 train_dataset=train_dataloader,
 eval_dataset=validation_dataloader,
)
trainer.train()


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1754353213322297>:16
      1 training_args = transformers.TrainingArguments(
      2 output_dir="idiot_save/",
      3 learning_rate=1e-3,
   (...)
      8 save_strategy='no',
      9 )
     10 trainer = transformers.Trainer(
     11  model=model,
     12  args=training_args,
     13  train_dataset=train_dataloader,
     14  eval_dataset=validation_dataloader,
     15 )
---> 16 trainer.train()

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-77857a59-781a-44e2-8ce5-ee2115b21373/lib/python3.9/site-packages/transformers/trainer.py:1858, in Trainer.train(self, resume_from_checkpoint, trial, ignore_keys_for_eval, **kwargs)
   1856         hf_hub_utils.enable_progress_bars()
   1857 else:
-> 1858     return inner_training_loop(
   1859         args=args,
   1860         resume_from_checkpoint=resume_from_checkpoint,
   1861    

In [ ]:
import time,random
device= 'cuda'
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch['input_ids'].to(device).squeeze()
        b_labels = batch['input_ids'].to(device).squeeze()
        b_masks = batch['attention_mask'].to(device).squeeze()

        model.zero_grad()        
        
        outputs = model(  input_ids=b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch['input_ids'].to(device).squeeze()
        b_labels = batch['input_ids'].to(device).squeeze()
        b_masks = batch['attention_mask'].to(device).squeeze()
        
        with torch.no_grad():        
            outputs = model(  input_ids=b_input_ids,
                              labels=b_labels, 
                              attention_mask = b_masks
                            )
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


---------------------------------------------------------------------------
OutOfMemoryError                          Traceback (most recent call last)
File <command-1489376779142648>:66
     62         model.train()
     64     loss.backward()
---> 66     optimizer.step()
     68     scheduler.step()
     70 # Calculate the average loss over all of the batches.

File /databricks/python/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:68, in _LRScheduler.__init__.<locals>.with_counter.<locals>.wrapper(*args, **kwargs)
     66 instance._step_count += 1
     67 wrapped = func.__get__(instance, cls)
---> 68 return wrapped(*args, **kwargs)

File /databricks/python/lib/python3.9/site-packages/torch/optim/optimizer.py:140, in Optimizer._hook_for_profile.<locals>.profile_hook_step.<locals>.wrapper(*args, **kwargs)
    138 profile_name = "Optimizer.step#{}.step".format(obj.__class__.__name__)
    139 with torch.autograd.profiler.record_function(profile_name):
--> 140     out = func(*arg

In [ ]:
train_dataloader.dataset.dataset.attn_masks[0].shape
train_dataloader.dataset.dataset.attn_masks[1].shape


Out[55]: torch.Size([111, 512])

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1489376779142649>:11
      1 training_args = transformers.TrainingArguments(
      2 output_dir="idiot_save/",
      3 learning_rate=1e-3,
   (...)
      8 save_strategy='no',
      9 )
     10 trainer = transformers.Trainer(
---> 11  model=model,
     12  args=training_args,
     13  train_dataset=train_dataloader,
     14  eval_dataset=validation_dataloader,
     15 )
     16 trainer.train()

NameError: name 'model' is not defined

In [ ]:
torch.cuda.empty_cache()